<a href="https://colab.research.google.com/github/erdemsirel/data_science_case/blob/main/DataScinceCaseThalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Source
# https://www.kaggle.com/code/juniorbueno/analyzing-credit-default/data

In [123]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from  sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Loading The Data

In [86]:
data = pd.read_csv("https://raw.githubusercontent.com/erdemsirel/data_science_case/main/credit_risk_dataset.csv")

In [87]:
def add_duedate_and_pay_date(row):
    np.random.seed(0)
    row['due_date'] = pd.to_datetime('2021-06-01') + pd.to_timedelta(np.random.randint(0,365), unit='D')
    if row['loan_status'] == 1:
        row['paid_date'] = row['due_date'] + pd.to_timedelta(np.random.randint(31,31*3), unit='D')
    else:
        row['paid_date'] = row['due_date'] + pd.to_timedelta(np.random.randint(0,29), unit='D')
    # Convert to string
    row['due_date'] = row['due_date'].strftime('%Y-%m-%d')
    row['paid_date'] = row['paid_date'].strftime('%Y%m%d')
    return row
data = data.apply(add_duedate_and_pay_date, axis=1)
data.loc[(data['loan_status'] == 1) & (data.index < 100), 'paid_date'] = pd.NaT

# Add label -1 cases
indeterminate = data.sample(n=138).copy()
indeterminate['loan_status'] = -1 
indeterminate['due_date'] = indeterminate['due_date'].apply(lambda x: (pd.to_datetime('2022-10-07') - pd.to_timedelta(np.random.randint(0,28), unit='D')).strftime('%Y-%m-%d'))
indeterminate['paid_date'] = None
print(data.shape)
data = pd.concat([data, indeterminate], axis='index').sample(frac=1).reset_index(drop=True)
print(data.shape)

(32581, 14)
(32719, 14)


# Exploratory Data Analysis

In [ ]:
data.isnull().sum() / data.count()

# Data Processing

## Building the label

In [88]:
data['due_date'] = pd.to_datetime(data['due_date'], format='%Y-%m-%d')
data['paid_date'] = pd.to_datetime(data['paid_date'], format='%Y%m%d')

In [95]:
mask_paid = (~data['paid_date'].isna())
mask_paid_date_30_days_past_due_date = (data['paid_date'] - data['due_date']).dt.days > 30
mask_current_date_30_days_past_due_date = (pd.to_datetime('2022-10-07') - data['due_date']).dt.days > 30

data.loc[mask_paid & mask_paid_date_30_days_past_due_date, "label"] = 1
data.loc[mask_paid & (~mask_paid_date_30_days_past_due_date), "label"] = 0
data.loc[(~mask_paid) & (mask_current_date_30_days_past_due_date), "label"] = 1
data.loc[(~mask_paid) & (~mask_current_date_30_days_past_due_date), "label"] = -1
data = data[data["label"] >= 0]


In [97]:
data[data['loan_status'] != data['label']]

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,due_date,paid_date,label


In [98]:
data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,due_date,paid_date,label
0,33,37440,RENT,5.0,PERSONAL,B,4400,11.71,0,0.12,N,10,2021-11-20,2021-12-05,0.0
1,35,49000,RENT,2.0,DEBTCONSOLIDATION,D,24000,15.20,1,0.49,N,10,2021-11-20,2022-02-06,1.0
2,24,130000,MORTGAGE,6.0,PERSONAL,B,20000,11.58,0,0.15,N,2,2021-11-20,2021-12-05,0.0
3,29,66500,OWN,4.0,VENTURE,A,12000,6.99,0,0.18,N,8,2021-11-20,2021-12-05,0.0
4,30,100000,MORTGAGE,0.0,PERSONAL,D,30000,16.77,0,0.30,Y,6,2021-11-20,2021-12-05,0.0


# Modelling

In [99]:
data = data.drop(columns=['due_date', 'paid_date', 'loan_status'])
data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,label
0,33,37440,RENT,5.0,PERSONAL,B,4400,11.71,0.12,N,10,0.0
1,35,49000,RENT,2.0,DEBTCONSOLIDATION,D,24000,15.20,0.49,N,10,1.0
2,24,130000,MORTGAGE,6.0,PERSONAL,B,20000,11.58,0.15,N,2,0.0
3,29,66500,OWN,4.0,VENTURE,A,12000,6.99,0.18,N,8,0.0
4,30,100000,MORTGAGE,0.0,PERSONAL,D,30000,16.77,0.30,Y,6,0.0


In [104]:
# data = pd.get_dummies(data, columns=["person_home_ownership", "loan_intent", "loan_grade", "cb_person_default_on_file"])
data.head(3).T

,0,1,2
person_age,33.00,35.00,24.00
person_income,37440.00,49000.00,130000.00
person_emp_length,5.00,2.00,6.00
loan_amnt,4400.00,24000.00,20000.00
loan_int_rate,11.71,15.20,11.58
loan_percent_income,0.12,0.49,0.15
cb_person_cred_hist_length,10.00,10.00,2.00
label,0.00,1.00,0.00
person_home_ownership_MORTGAGE,0.00,0.00,1.00
person_home_ownership_OTHER,0.00,0.00,0.00


In [137]:
data = data.dropna()
x = data.drop(columns='label')

y = data['label']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=1)

In [138]:
model = RandomForestClassifier()

In [139]:
model.fit(x_train, y_train)

RandomForestClassifier()

In [140]:
print(metrics.classification_report(y_test, model.predict(x_test)))
print("AUC:", metrics.roc_auc_score(y_test, model.predict_proba(x_test)[:,1]))

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      7437
         1.0       0.97      0.71      0.82      2014

    accuracy                           0.93      9451
   macro avg       0.95      0.85      0.89      9451
weighted avg       0.94      0.93      0.93      9451

AUC: 0.9232128161895907


In [147]:
pd.Series(model.feature_importances_, index=x.columns).sort_values(ascending=False)

loan_percent_income               0.228596
person_income                     0.132211
loan_int_rate                     0.128437
loan_amnt                         0.073536
person_emp_length                 0.059369
person_home_ownership_RENT        0.053536
loan_grade_D                      0.047058
person_age                        0.044659
cb_person_cred_hist_length        0.035193
person_home_ownership_MORTGAGE    0.025109
loan_intent_DEBTCONSOLIDATION     0.022630
loan_grade_C                      0.021216
loan_intent_MEDICAL               0.021183
person_home_ownership_OWN         0.017183
loan_intent_HOMEIMPROVEMENT       0.013046
loan_grade_E                      0.012544
loan_intent_EDUCATION             0.011171
loan_intent_PERSONAL              0.009420
loan_grade_A                      0.009102
loan_intent_VENTURE               0.007942
cb_person_default_on_file_N       0.007724
cb_person_default_on_file_Y       0.006730
loan_grade_B                      0.006517
loan_grade_